# Feasiblity Check

In [2]:
!pip install -q transformers

In [62]:
from huggingface_hub import notebook_login

notebook_login()


## kf-deberta example

In [63]:
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained("kakaobank/kf-deberta-base")
tokenizer = AutoTokenizer.from_pretrained("kakaobank/kf-deberta-base")

text = "카카오뱅크와 에프엔가이드가 금융특화 언어모델을 공개합니다."
tokens = tokenizer.tokenize(text)
print(tokens)

inputs = tokenizer(text, return_tensors="pt")
model_output = model(**inputs)
print(model_output)

['카카오', '##뱅크', '##와', '에프', '##엔', '##가이', '##드', '##가', '금융', '##특', '##화', '언어', '##모델', '##을', '공개', '##합니다', '.']
BaseModelOutput(last_hidden_state=tensor([[[ 0.3266,  1.0502, -0.6938,  ...,  1.5691, -1.0747,  1.1697],
         [-0.3798,  2.1233,  0.3641,  ..., -0.1863, -0.9263, -0.7637],
         [ 0.6740,  1.2344, -0.6436,  ..., -0.5377, -1.3844,  0.1552],
         ...,
         [ 1.3960, -0.5337, -1.5322,  ..., -0.4973,  0.5771, -0.4824],
         [ 1.7078, -0.1235, -1.4388,  ...,  1.7612,  0.1432, -1.7462],
         [ 0.2516,  0.9185, -0.9216,  ...,  1.8321, -1.4633,  0.3245]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)


# sentiment analysis pipeline

In [88]:
from transformers import pipeline

checkpoint = "kakaobank/kf-deberta-base"
classifier = pipeline("sentiment-analysis", model=checkpoint)
classifier(
    [
        "신세계백화점이 지난해 역대 최대 매출을 올렸다.",
        "국내 최대 컨테이너 선사인 HMM 매각 협상이 6일 최종 결렬되면서 산업은행이 고심에 빠졌다.",
    ]
)


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at kakaobank/kf-deberta-base and are newly initialized: ['classifier.weight', 'pooler.dense.bias', 'classifier.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'label': 'LABEL_1', 'score': 0.5270591974258423},
 {'label': 'LABEL_0', 'score': 0.5600322484970093}]

처음에는 0번 라벨이 긍정, 1번이 부정으로 추정했다. 그래서 전처리, 모델 입력, 후처리로 나눠보았다.

## 토크나이저로 전처리



In [89]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(checkpoint)


In [90]:
raw_inputs = [
    "신세계백화점이 지난해 역대 최대 매출을 올렸다.",
    "국내 최대 컨테이너 선사인 HMM 매각 협상이 6일 최종 결렬되면서 산업은행이 고심에 빠졌다.",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[     2,  10630, 125995,    405,   2371,    598,    424, 126000,   3036,
         125996,      6,      3,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0],
        [     2,    332,    598,   5692,   4099, 126018,  22872,   1613,   1497,
         125995,    128, 126027,   1468,  13689, 126097,  20152,   5877, 125995,
          10949, 125999,   5856, 125996,      6,      3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


## 모델 살펴보기

### 고차원 벡터

In [91]:
from transformers import AutoModel

model = AutoModel.from_pretrained(checkpoint)

In [92]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([2, 24, 768])


### 모델 헤드

In [93]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at kakaobank/kf-deberta-base and are newly initialized: ['classifier.weight', 'pooler.dense.bias', 'classifier.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [94]:
print(outputs.logits.shape)

torch.Size([2, 2])


## 후처리

In [95]:
print(outputs.logits)

tensor([[ 0.2144, -0.0489],
        [ 0.1741,  0.0104]], grad_fn=<AddmmBackward0>)


In [96]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[0.5654, 0.4346],
        [0.5408, 0.4592]], grad_fn=<SoftmaxBackward0>)


In [97]:
model.config.id2label

{0: 'LABEL_0', 1: 'LABEL_1'}

##결과

1. 감성분석 파이프파인을 실행하면, 몇몇 웨이트가 초기화됨. 학습이 필요.
2. 레이블의 의미가 긍정, 부정으로 나오지 않음. 번호로 나와서 의미가 확실하지 않음.
3. 여러번 실행해보면, 그때마다 숫자가 다르게 나와서 스레숄드 확인이 어려움.

DB로 사용하여 학습하고, 정확한 스레숄드의 확인이 필요.